In [1]:
import os 

In [2]:
%pwd

'/home/ahmed/project/Kidney-Disease-Classification-Deep-learning-project/recsearch'

In [3]:
os.chdir('../')


In [4]:
%pwd

'/home/ahmed/project/Kidney-Disease-Classification-Deep-learning-project'

In [5]:
# training:
#   root_dir: artifacts/training
#   trained_model_path: artifacts/training/model.h5


In [6]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    update_base_model: Path 
    training_data: Path
    param_image_size: list
    param_batch_size: int
    param_epochs: int 
    params_augmentation: bool # import in param.yaml
    param_learning_rate: float



In [7]:
from project.constants import *
from project.utils import create_directories,read_yaml

In [8]:
class ConfigerationManager:
    def __init__(self, config=CONFIG_YAML_FILE, param=PARAM_YAML_FILE):
        self.config = read_yaml(config)
        self.param = read_yaml(param)
       
        create_directories(self.config.artifacts_root)
    
    def get_training_config(self):
        training = self.config.training 
        prepare_base_model = self.config.prepare_base_model
        trainig_data = os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")
        create_directories(training.root_dir)

        training_config = TrainingConfig(
            root_dir= training.root_dir, 
            trained_model_path=training.trained_model_path, 
            update_base_model=prepare_base_model.update_base_model, 
            training_data=trainig_data, 
            param_image_size=self.param.IMAGE_SIZE, 
            param_batch_size=self.param.BATCH_SIZE, 
            param_epochs=self.param.EPOCHS, 
            params_augmentation = self.param.AUGMENTATION,
            param_learning_rate= self.param.LEARNING_RATE
        )
        return training_config



In [9]:
import tensorflow as tf
import math

2025-11-24 11:58:02.755382: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-24 11:58:03.450247: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-24 11:58:06.593664: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [ ]:
import math
from pathlib import Path
import tensorflow as tf


class Training:
    def __init__(self, config:TrainingConfig):
        """
        Args:
            config: Instance of TrainingConfig containing all training parameters.
        """
        self.config = config
        

    def get_base_model(self):
        """Load and compile the base model."""
        self.model = tf.keras.models.load_model(self.config.update_base_model)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.param_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        """Create training and validation generators with optional augmentation."""
        # Validation generator
        val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2
        )

        self.val_data = val_datagen.flow_from_directory(
            directory=self.config.training_data,
            target_size=self.config.param_image_size[:-1],  # (H, W)
            batch_size=self.config.param_batch_size,
            interpolation="bilinear",
            shuffle=False,
            subset='validation'
        )

        # Training generator with optional augmentation
        if self.config.params_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split=0.2,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True
            )
        else:
            train_datagen = val_datagen

        self.train_data = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            target_size=self.config.param_image_size[:-1],
            batch_size=self.config.param_batch_size,
            interpolation="bilinear",
            shuffle=True,
            subset='training'
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Save the trained model to the given path."""
        model.save(str(path))

    def train(self, callbacks: list = None):
        """
        Train the model using the training and validation generators.

        Args:
            callbacks (list, optional): Keras callbacks for training.
        """
        # Calculate steps to cover all samples
        steps_per_epoch = math.ceil(self.train_data.samples / self.train_data.batch_size)
        validation_steps = math.ceil(self.val_data.samples / self.val_data.batch_size)

        # Train the model
        history = self.model.fit(
            self.train_data,
            epochs=self.config.param_epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=self.val_data,
            validation_steps=validation_steps,
            callbacks=callbacks,
            verbose=1
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        return history


In [11]:

try:
    # Initialize configuration manager
    config = ConfigerationManager()  

    # Prepare callbacks (optional, uncomment if needed)
    # callbacks_config = config.get_prepare_callback_config()
    # callbacks = PrepareCallback(config=callbacks_config).get_tb_ckpt_callbacks()

    # Training setup
    training_config = config.get_training_config()
    trainer = Training(config=training_config)  # fixed spelling from 'Traning'

    # Prepare model and data
    trainer.get_base_model()
    trainer.train_valid_generator()
    trainer.train()

    # Train the model (uncomment when callbacks are ready)
    # trainer.train(callbacks=callbacks)

except Exception as e:
    raise e


2025-11-24 11:58:09.538191: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/2
24/24 ━━━━━━━━━━━━━━━━━━━━ 104s 4s/step - accuracy: 0.7527 - loss: 7.1753 - val_accuracy: 0.6129 - val_loss: 3.8068
Epoch 2/2
24/24 ━━━━━━━━━━━━━━━━━━━━ 104s 4s/step - accuracy: 0.8871 - loss: 1.5820 - val_accuracy: 1.0000 - val_loss: 0.0066
